In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

# Astronomy
import astropy
from astropy.units import au, minute

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_elements, load_ast_vectors
from asteroid_spline import get_df_shape, spline_ast_data, make_spline_df
from asteroid_direction import calc_dir_ast2obs
from astro_utils import deg2dist, dist2sec
from orbital_element import elt2pos
from planets_interp import get_sun_pos
from db_utils import sp2df, df2db

In [3]:
from detection_near_ast import get_data_detections, get_data_ast_dir, asteroid_batch_prelim, asteroid_batch_vec, asteroid_batch_elt, light_time_adj
from detection_near_ast import cols_u_obs, cols_q_obs, cols_u_ast, cols_v_ast, cols_q_ast, mjd0, mjd1, c

In [4]:
# Inputs
d0 = 901
d1 = 905
n0 = 1
n1 = 3
arcmin_max = 60*160

In [5]:
# Get detections
det = get_data_detections(d0=d0, d1=d1)
det

,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z
DetectionID,,,,,,,,
901,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032
902,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032
903,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032
904,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032


In [6]:
# Get asteroid directions
ast_in = get_data_ast_dir(n0=n0, n1=n1)
ast_in

,AsteroidID,TimeID,mjd,uAst_x,uAst_y,uAst_z,LightTime
0,1,83888640,58256.0,-0.695441,0.695787,0.179561,21.312345
1,1,83894400,58260.0,-0.710843,0.680749,0.176872,21.715483
2,1,83900160,58264.0,-0.726426,0.664782,0.174268,22.114047
3,1,83905920,58268.0,-0.742106,0.647905,0.171747,22.507469
4,1,83911680,58272.0,-0.757805,0.630133,0.169305,22.895268
...,...,...,...,...,...,...,...
525,2,85386240,59296.0,0.876943,-0.416402,0.239959,34.756723
526,2,85392000,59300.0,0.886408,-0.397316,0.237530,34.511296
527,2,85397760,59304.0,0.895327,-0.378259,0.235178,34.245183
528,2,85403520,59308.0,0.903706,-0.359268,0.232898,33.958757


In [7]:
# First pass at detections near asteroids
dna_pre = asteroid_batch_prelim(det=det, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [8]:
dna_pre

,AsteroidID,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s
0,1,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.168125,22.718397,-0.750618,0.638382,0.170414,0.725050
1,1,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.168125,22.718397,-0.750618,0.638382,0.170414,0.720252
2,1,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.169965,22.718575,-0.750625,0.638374,0.170413,0.740616
3,1,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.169965,22.718575,-0.750625,0.638374,0.170413,0.740559
4,2,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.168125,24.277859,-0.096585,0.916142,-0.389044,1.479665
5,2,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.168125,24.277859,-0.096585,0.916142,-0.389044,1.477228
6,2,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.169965,24.277919,-0.096603,0.916141,-0.389041,1.477603
7,2,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.169965,24.277919,-0.096603,0.916141,-0.389041,1.477601


In [9]:
# Sharpen estimate using splined asteroid vectors
dna_vec = dna_pre.copy()
asteroid_batch_vec(dna=dna_vec, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [10]:
dna_vec

,AsteroidID,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s,qAst_x,qAst_y,qAst_z
0,1,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.152365,22.694751,-0.750298,0.638714,0.170580,0.725373,-2.389674,0.795610,0.465448
1,1,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.152365,22.694751,-0.750298,0.638714,0.170580,0.720572,-2.389674,0.795610,0.465448
2,1,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.154205,22.694932,-0.750305,0.638706,0.170579,0.740975,-2.389680,0.795591,0.465449
3,1,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.154205,22.694932,-0.750305,0.638706,0.170579,0.740918,-2.389680,0.795591,0.465449
4,2,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.151272,24.267738,-0.095538,0.916171,-0.389233,1.480364,-0.621033,1.726017,-1.135789
5,2,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.151272,24.267738,-0.095538,0.916171,-0.389233,1.477927,-0.621033,1.726017,-1.135789
6,2,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.153113,24.267801,-0.095555,0.916171,-0.389230,1.478314,-0.621055,1.726011,-1.135783
7,2,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.153113,24.267801,-0.095555,0.916171,-0.389230,1.478312,-0.621055,1.726011,-1.135783


In [11]:
# Report light time error
_  = light_time_adj(df=dna_vec, verbose=True)

Mean light time error: 3.86e-09 minutes


In [14]:
# Sharpen estimate using splined asteroid elements
dna_elt = dna_pre.copy()
asteroid_batch_elt(dna=dna_elt, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [16]:
dna_elt

,AsteroidID,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s,qAst_x,qAst_y,qAst_z
0,1,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.152365,22.694751,-0.750298,0.638714,0.170580,0.725373,-2.389674,0.795610,0.465448
1,1,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.152365,22.694751,-0.750298,0.638714,0.170580,0.720572,-2.389674,0.795610,0.465448
2,1,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.154205,22.694932,-0.750305,0.638706,0.170579,0.740975,-2.389680,0.795591,0.465449
3,1,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.154205,22.694932,-0.750305,0.638706,0.170579,0.740918,-2.389680,0.795591,0.465449
4,2,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032,58270.151272,24.267738,-0.095538,0.916171,-0.389233,1.480364,-0.621033,1.726017,-1.135789
5,2,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032,58270.151272,24.267738,-0.095538,0.916171,-0.389233,1.477927,-0.621033,1.726017,-1.135789
6,2,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032,58270.153113,24.267801,-0.095555,0.916171,-0.389230,1.478314,-0.621055,1.726011,-1.135783
7,2,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032,58270.153113,24.267801,-0.095555,0.916171,-0.389230,1.478312,-0.621055,1.726011,-1.135783


In [17]:
# Direction from three methods
u_pre = dna_pre[cols_u_ast].values
u_vec = dna_vec[cols_u_ast].values
u_elt = dna_elt[cols_u_ast].values

In [18]:
# Difference in direction 
def direction_diff(u0: np.ndarray, u1: np.ndarray):
    du = u1 - u0
    delta = np.sqrt(np.sum(np.square(du), axis=-1))
    # Change in arc seconds
    mean_delta_sec = dist2sec(np.mean(delta))
    return mean_delta_sec

In [19]:
# Difference between methods
delta_pre_vec = direction_diff(u0=u_pre, u1=u_vec)
delta_pre_elt = direction_diff(u0=u_pre, u1=u_elt)
delta_vec_elt = direction_diff(u0=u_vec, u1=u_elt)

In [22]:
# Report results
print(f'Mean change in direction between different methods:')
print(f'pre vs. vec: {delta_pre_vec:5.2f} arc seconds')
print(f'pre vs. elt: {delta_pre_elt:5.2f} arc seconds')
print(f'vec vs. elt: {delta_vec_elt:5.2e} arc seconds')


Mean change in direction between different methods:
pre vs. vec: 160.38 arc seconds
pre vs. elt: 160.38 arc seconds
vec vs. elt: 8.37e-05 arc seconds
